# Programming language interoperability

## Python

In [ ]:
using PyCall

In [ ]:
np = pyimport("numpy")

In [ ]:
np.linalg.eigvals(np.random.rand(5,5))

In [ ]:
M = rand(5,5)
np.linalg.eigvals(M)

In [ ]:
py"""
import numpy as np

def sinpi(x):
    return np.sin(np.pi * x)
"""

In [ ]:
py_sinpi(x) = py"sinpi"(x)

In [ ]:
py_sinpi(10)

In [ ]:
using BenchmarkTools
@btime py_sinpi(10);

## C

In [ ]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

Compile to a shared library by piping `c_code` to gcc:

In [ ]:
using Libdl
const Clib = tempname()

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

Binding the function from the shared library:

In [ ]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

In [ ]:
c_sum(rand(10))

In [ ]:
x = rand(10)
@btime c_sum($x);

## Mixing Julia, Python, and C

Julia (`real`), Python/numpy (`py_sinpi`), C (`c_sum`)

In [ ]:
x = rand(10);

In [ ]:
py_sinpi(real(c_sum(x)))

In [ ]:
@btime py_sinpi(real(c_sum($x)));

See [JuliaInterop](https://github.com/JuliaInterop) for more, such as [RCall.jl](https://github.com/JuliaInterop/RCall.jl), [JavaCall.jl](https://github.com/JuliaInterop/JavaCall.jl), and [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).